In [1]:
from tqdm import tqdm
import glob
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# 데이터 전처리 과정
def preprocessing():
    
    data = []
    datatype = ["interactive", "bulk", "video", "web"]
    
    # datatype별로 순회
    for dt in tqdm(range(len(datatype))):
        sum_df = []

        # 각 타입의 각 csv 파일을 하나씩 불러옵니다.
        for fn in glob.glob(f'./train/{datatype[dt]}_*.csv'):

            df = pd.read_csv(fn)
            # 시간 변경
            df = df[['time', 'data_len', 'ip_dst']]
            df['time'] = df['time'].apply(lambda x : datetime.fromtimestamp(x).strftime('%Y/%m/%d %H:%M:%S'))

            # time 컬럼과 ip_dst 컬럼을 groupby를 통해 그룹별 분할하여 해당 일자에 대한 data_len 컬럼의 합을 도출
            new_df = df.groupby(['time', 'ip_dst']).count().reset_index()

            # groupby한 ip_dst의 데이터 개수가 40개 이하인 경우, 해당 ip 데이터는 제거
            unique_ip = new_df.groupby('ip_dst').count()[new_df.groupby('ip_dst').count()['data_len']>40].index

            # unique_ip가 존재하지 않을 경우 넘어감
            if unique_ip.empty:
                continue

            # ip_dst, time순으로 데이터프레임 정렬
            new_df2 = new_df[new_df['ip_dst'].isin(unique_ip)].sort_values(['ip_dst','time']).reset_index(drop=True)

            # 각 ip_dst에 대해 11개의 크기로 데이터셋을 window 분할
            x_values = []
            ip_values = []
            for u_ip in unique_ip:
                a = new_df2[new_df2['ip_dst']==u_ip]['data_len'].values
                for i in np.arange(0, a.shape[0]+1-11): 
                    x_values.append(a[i : i+11]) # size 11로 windowing한 값 x_values에 저장
                    ip_values.append(u_ip)
          
            # 분할된 window들을 dataframe으로 만듦
            df2 = pd.DataFrame(x_values)

            # column의 이름을 trafic(t-x)로 변경
            col = [f'traffic(t-{x})' for x in range(11)]
            col.reverse()
            df2.columns = col
            df2 = df2.rename(columns={'traffic(t-0)':'traffic(t)'})

            # label 추가
            df2["target"] = dt

            # sum_df에 각 ip_dst에 따른 데이터들 저장
            sum_df.append(df2)

        # 모든 ip_dst에 대해 데이터 concat한 결과
        sum_df = pd.concat(sum_df, axis=0)
        data.append(sum_df)
    
    # 모든 데이터타입 데이터프레임 합침
    result_df = pd.concat(data, axis=0)
    
    return result_df

# 피쳐앤지니어링
def make_feature(train):
    """
    row 단위로 피쳐엔지니어링
    axis=1 방향으로 (전체 column에 대해 1개의 row를) mean, sum, max, median한 값을 새로운 feature로 추가
    """
    all_df = []
    train2 = train.copy()
    all_df.append(train2)

    temp_df = train2.drop(['target'], axis=1)
    all_df.append(pd.DataFrame(temp_df.mean(1)).rename(columns={0:'mean'}))
    all_df.append(pd.DataFrame(temp_df.sum(1)).rename(columns={0:'sum'}))
    all_df.append(pd.DataFrame(temp_df.max(1)).rename(columns={0:'max'}))
    all_df.append(pd.DataFrame(temp_df.median(1)).rename(columns={0:'median'}))
        
    return all_df

train = preprocessing() # 전처리된 기본 데이터
test = pd.read_csv('./test.csv') # 기본 테스트 데이터
test['target'] = -1 # train에서 target행을 따로 만들어줬으므로 test에도 똑같이 column생성해줌

# 피쳐앤지니어링 이후 만들어진 데이터셋
new_train = pd.concat(make_feature(train), axis=1)
new_test = pd.concat(make_feature(test), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.16s/it]


In [2]:
class Node:
    def __init__(self, num_samples, num_samples_per_class, predicted_class):
        self.num_samples = num_samples # sample 개수
        self.num_samples_per_class = num_samples_per_class # 각 class별 sample
        self.predicted_class = predicted_class # 예측 class
        self.feature_index = 0
        self.threshold = 0
        self.left = None # left 자식
        self.right = None # right 자식
        
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth # tree의 max_depth 지정 
        self.tree = None
        self.n_features = 0
        self.n_classes = 0
    
    def fit(self, x, y):
        self.n_classes = len(np.unique(y)) # class의 개수는 unique한 y의 개수
        self.n_features = x.shape[1] # dataset의 길이 (개수)
        self.tree = self.build_tree(x, y) # build_tree 실행
    
    def predict_proba(self, x):
    # 단순 predicted class가 나오는 것이 아니라 각 class별 probability가 나오도록 함
        prob_list = []
        for inputs in  x:
            node = self.tree
            
            # node의 feature_index 값이 threshold 이상이 될때까지 순회 (right node)
            while node.left:
                if inputs[node.feature_index] < node.threshold:
                    node = node.left
                else:
                    node = node.right
            
            # sample들의 n개 class별 probability들을 저장
            proba = [float(node.num_samples_per_class[i]) / node.num_samples if node.num_samples != 0 else 0.0 for i in range(self.n_classes)]
            prob_list.append(proba)
        
        # 확률값 추출
        return prob_list
    
    
    def best_split(self, x, y):
        
        """
        Node에 가장 적합한 split을 찾기 위함.
        "Best"는 모집단에 의해 가중치가 부여된 두 자녀의 평균 impurity가 가장 작다는 것을 의미하며 현재 노드의 impurity보다 작아야함
        best split을 찾기 위해 모든 feature들을 loop으로 훑고 
        인접한 training samples 사이의 모든 midpoints을 가능한 threshold값으로 고려.
        특정 feature/threshold 쌍에 의해 생성된 split의 Gini impurity를 계산하고 impurity가 가장 적은 쌍을 반환
        
        """
        # None으로 초기화
        best_feature_index = None
        best_threshold = None 
        
        # node를 split하려면 두개 이상의 요소가 필요
        if y.size > 1:

            num_parent = [np.sum(y == c) for c in range(self.n_classes)] # 현재 node안에 있는 각 클래스 수를 계산
            
            # 현재 node의 gini
            bg = 0
            for parent in num_parent:
                bg += (parent/y.size)**2
            best_gini = 1.0 - bg
            
            # 모든 feature를 for loop으로 탐색
            for feature_index in range(self.n_features):
                
                thresholds, classes = zip(*sorted(zip(x[:, feature_index], y))) # 선택된 feature index에 따라 threshold와 class를 sort

                # feature/threshold 쌍에 따라 node를 split하고 children의 각 class에 대한 결과 모집단을 계산
                num_left = [0] * self.n_classes
                num_right = num_parent.copy()

                for i in range(1, y.size): # split 가능한 경우의 수
                    
                    # gini계수 left, right 업데이트
                    gl, gr = 0, 0
                    for p in range(self.n_classes):
                        gl += (num_left[p] / i) ** 2
                        gr += (num_right[p] / (y.size - i)) ** 2
                    
                    # node의 개수 증가/감소
                    num_left[classes[i-1]] += 1
                    num_right[classes[i-1]] -= 1
                    
                    # 1.0-gl = left gini
                    # 1.0-gr = right gini
                    # 자식 gini impurity의 weighted average
                    gini = (i * (1.0-gl) + (y.size - i) * (1.0-gr)) / y.size

                    # 해당 feature에 대해 동일한 값을 가지는 구간을 나누지 않기 위한 조건 
                    # (둘다 split의 같은 쪽에 있어야하므로)
                    if thresholds[i] != thresholds[i-1] and gini < best_gini:
                        best_gini = gini
                        best_feature_index = feature_index # best split을 위한 feature의 index
                        best_threshold = (thresholds[i] + thresholds[i - 1]) / 2 # split에 사용하기 위한 최상의 threshold
        
        return best_feature_index, best_threshold 

    def build_tree(self, x, y, depth=0):
        """
        recursive하게 best split을 찾아 decision tree 구축
        
        """
        node = Node(num_samples=len(y), # node정의
                    num_samples_per_class=[np.sum(y == i) for i in range(self.n_classes)], # 현재 node의 각 class들의 모집단
                    predicted_class=np.argmax([np.sum(y == i) for i in range(self.n_classes)]))  # 가장 모집단이 많은 class
        
        # max_depth이전까지 recursive하게 split한다. 
        # depth가 max depth보다 깊어지면 멈춤 (stopping criteria)
        if depth < self.max_depth:
            feature_index, threshold  = self.best_split(x, y)
            if threshold is not None:
                idx = x[:, feature_index] <= threshold
                node.feature_index = feature_index
                node.threshold = threshold
                node.left = self.build_tree(x[idx], y[idx], depth + 1)
                node.right = self.build_tree(x[~idx], y[~idx], depth + 1)
                
        return node

In [3]:
def predict_decision_kfold(train_df, test_df):
    
    train = train_df.copy() 
    train = train.reset_index(drop=True) # 입력받은 데이터셋 인덱스 초기화
    skf = StratifiedKFold(n_splits=5, shuffle=True) # stratified kfold 사용 (5 fold)
    train['fold'] = -1 # fold 초기화
    excluded_features = ['target', 'fold'] # train에 사용하지 않을 2가지 column들 제거
    col = [x for x in train.columns if x not in excluded_features] # train에 사용할 feature만 추출
    
    y = train['target'].copy() # y에 train target값 저장
    oof = np.zeros((train.shape[0], y.nunique()))  # shape = [train data 개수, class (4가지 datatype)]
    preds = np.zeros((test_df.shape[0], y.nunique())) # shape = [test data 개수, class (4가지 datatype)]
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train,train['target'])): # skf로 index 분할 (train, val)
        trn_x, trn_y = train[col].iloc[train_idx], y.iloc[train_idx]  # training data
        val_x, val_y = train[col].iloc[val_idx], y.iloc[val_idx] # validation data

        decision_tree = DecisionTree(max_depth = 8) # decision tree
        decision_tree.fit(trn_x.values, trn_y.values.reshape(-1)) # 학습
        
        y_pred = decision_tree.predict_proba(val_x.values) # validation 예측
        print(f'val {fold} fold acc:', accuracy_score(np.array(y_pred).argmax(1), val_y)) # validation 예측결과 print
        
        oof[val_idx] = y_pred # validation 예측결과 저장
        
        preds += np.array(decision_tree.predict_proba(test_df[col].values))/5 # test data에 대해 예측 (5 fold이므로 /5)
        
    print('CV ACC :', accuracy_score(oof.argmax(1), train['target'])) # argmax를 통해 가장 확률값이 높은 class를 예측값으로 지정 후 acc 측정
    
    return preds, oof


preds = 0 # test prediction
val_preds=0 # validation prediction
iter_ = 20

# iteration 만큼 과정 반복
for i in range(iter_):
    preds_, val_preds_ = predict_decision_kfold(new_train, new_test)
    preds += preds_/iter_
    val_preds += val_preds_/iter_

val 0 fold acc: 0.9367338869118228
val 1 fold acc: 0.9501779359430605
val 2 fold acc: 0.9434559114274417
val 3 fold acc: 0.9529458283906682
val 4 fold acc: 0.9485962831158561
CV ACC : 0.9463819691577698
val 0 fold acc: 0.9422696718070384
val 1 fold acc: 0.9493871095294583
val 2 fold acc: 0.9517595887702649
val 3 fold acc: 0.9454329774614472
val 4 fold acc: 0.9509687623566627
CV ACC : 0.9479636219849743
val 0 fold acc: 0.9470146302886516
val 1 fold acc: 0.9505733491498616
val 2 fold acc: 0.9442467378410438
val 3 fold acc: 0.9470146302886516
val 4 fold acc: 0.9474100434954528
CV ACC : 0.9472518782127323
val 0 fold acc: 0.9478054567022538
val 1 fold acc: 0.9470146302886516
val 2 fold acc: 0.9478054567022538
val 3 fold acc: 0.9418742586002372
val 4 fold acc: 0.9541320680110715
CV ACC : 0.9477263740608937
val 0 fold acc: 0.9501779359430605
val 1 fold acc: 0.9482008699090549
val 2 fold acc: 0.9474100434954528
val 3 fold acc: 0.9450375642546461
val 4 fold acc: 0.9422696718070384
CV ACC : 0.94

In [4]:
print('CV ACC :', accuracy_score(val_preds.argmax(1), train['target'])) # train에 대한 성능 측정 (리더보드 이전)
# CV ACC : 0.9517595887702649
# CV ACC : 0.9529458283906682

CV ACC : 0.9522340846184263


In [5]:
sub = pd.read_csv('./sample_submission.csv')
sub['type'] = preds.argmax(1)
sub # 첫번째 예측값

,id,type
0,1,3
1,2,0
2,3,0
3,4,0
4,5,3
...,...,...
3135,3136,3
3136,3137,0
3137,3138,0
3138,3139,0


In [6]:
""" 꼬리찾기 postprocessing (꼬리는 sequence의 시작을 찾기위함)"""

temp = test.drop('target', axis=1) # test의 target column없애기
temp['id'] = np.arange(temp.shape[0]) # index를 id라고 지정
temp= temp.set_index('id')

a=0
temp['order'] = -1 # order column 생성
all_df = []
id_set = set()

for i in tqdm(range(temp.shape[0])):
    # 첫번째 row부터 꼬리찾기 시작
    temp_df = []
    tail = temp.iloc[[i], :].copy() # i번째 row를 tail이라고 부름
    
    # 새로운 꼬리를 찾고, 있으면 넘어가기
    if tail.index[0] in id_set:
        continue
    
    # tail은 계속 temp_df와 id_set에 추가됨
    id_set.add(tail.index[0])
    temp_df.append(tail)
    
    while True:
        # tail 의 첫번째와 모든 temp랑 비교를 해본다.
        # tail.iloc[0,1:]은 해당 꼬리의 t-10을 제외한 t-9부터 t 까지의 값을 의미
        # temp.iloc[:,:-1]은 t를 제외한 t-10부터 t-1 까지의 값을 의미
        # 두 값을 비교해서 10개의 값이 모두 같으면 sequence가 될 수 있는 부분이 있다는 것 -> idx_에 저장 
        idx_ = (tail.iloc[0,1:].values == temp.iloc[:,:-1].values).sum(1) == 10
        
        # 그래서 꼬리를 찾기 시작, 꼬리가 마무리 될때까지
        # (temp는 iloc이나 loc이나 똑같음)
        if temp.loc[idx_].shape[0] == 1: # 만약 꼬리를 1개만 찾았다면
            tail = temp.loc[idx_].copy()
            id_set.add(tail.index[0]) # 방문한적 있다고 추가
            temp_df.append(tail) # tail을 모으는 곳에 추가 (마지막 tail)
            
        elif temp.loc[idx_].shape[0] == 0:
            all_df.append(temp_df) # 만약 0개가 됐으면, 그동안 모은 temp_df 다 all_df에 넣고 마무리
            break
            
        elif temp.loc[idx_].shape[0] > 1: # 만약 꼬리가 여러개가 찾아진다면

            # 그냥 꼬리 하나씩 찾아주기
            tail = temp.loc[idx_].copy()
            tail_id = [x for x in tail.index if x not in id_set] # tail이 id_set에 존재하지 않으면 tail_id에 저장
            id_set.update(tail.index.tolist()) # tail 인덱스 id_set에 업데이트
            
            if len(tail_id)==0: # tail_id가 id_set에 이미 다 있으면 temp_df 다 all_df에 넣고 마무리
                all_df.append(temp_df)
                break
            
            # 새로운 tail을 선정 
            tail = tail.loc[[tail_id[0]], :].copy()


100%|█████████████████████████████████████████████████████████████████████████████| 3140/3140 [00:07<00:00, 423.36it/s]


In [7]:
"""
위에있는 경우는, 중복이 많음
index순서대로 보면

[0,3,4,6,8,11,13] -> 이게 하나의 sequence인줄 알았는데
[110, 23, 44, 11,  0,3,4,6,8,11,13] -> 사실은 sequcne의 시작이 110 index부터 였던 경우

이렇게 중복되는 경우가 다 들어가있으니까, 중복이 많은 상태

1. 그래서 여기서는 인덱스 0부터 ~3140 (테스트수) 까지 for구문 돌려서
2. 위에서 만든 리스트의 데이터프레임 하나하나 조회
- 예를들어서 인덱스 0을 포함하는 모든 데이터프레임 찾은다음에,, 그중에서 가장 긴 경우 뽑음
3. 그래서 가장 긴 데이터프레임 하나만 선택
"""

all_df2=[]
all_f_df = []
id_set = set()

# 데이터프레임 index초기화
for i in range(len(all_df)):
    f_df = pd.concat(all_df[i]).reset_index()
    all_f_df.append(f_df)

for j in tqdm(np.arange(test.shape[0])):
    if j in id_set:
        continue
    for f_df in all_f_df: # 위에서 만들어진 데이터프레임 조회
        max_=0
        if j in f_df.id.tolist():
            id_set.update(f_df.id.tolist())
            
            if f_df.shape[0] > max_: # 인덱스를 포함하는 데이터프레임을 찾고 그중에서 가장 긴 sequence를 뽑음
                max_f_df = f_df.copy()
                max_ = f_df.shape[0]
                
    all_df2.append(max_f_df) # 가장 긴 데이터프레임 하나만 선택


100%|████████████████████████████████████████████████████████████████████████████| 3140/3140 [00:00<00:00, 6451.09it/s]


In [8]:
"""
테스트 불러오는 단계
"""

sub2 = sub.copy() # 첫번째 예측값
sub2
test = pd.read_csv('./test.csv')
test['target'] = sub2['type'] # test의 target부분에 예측값을 넣음
test['id'] = np.arange(test.shape[0])

In [9]:
"""
1. csv 각각을 분리
2. 각각 csv들 중에 하나의 클래스만으로 통일 안된애들 찾는과정
"""
a=[]
for i in range(len(all_df2)):
    temp2 = all_df2[i]
    if pd.merge(temp2, test[['id', 'target']], how='left', on='id')['target'].value_counts().shape[0]!=1:
        a.append(i)

In [10]:
len(a) # 하나의 클래스로 통일 안된 csv가 10개라는뜻

10

In [11]:
a

[0, 4, 10, 22, 28, 31, 34, 39, 40, 42]

In [12]:
# sequence나오는거 확인
all_df2[0]

,id,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),order
0,686,99.0,22.0,18.0,264.0,182.0,1335.0,1316.0,1439.0,1677.0,149.0,8,-1
1,590,22.0,18.0,264.0,182.0,1335.0,1316.0,1439.0,1677.0,149.0,8.0,2,-1
2,203,18.0,264.0,182.0,1335.0,1316.0,1439.0,1677.0,149.0,8.0,2.0,3,-1
3,2714,264.0,182.0,1335.0,1316.0,1439.0,1677.0,149.0,8.0,2.0,3.0,1,-1
4,959,182.0,1335.0,1316.0,1439.0,1677.0,149.0,8.0,2.0,3.0,1.0,582,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,2660,7.0,6.0,12.0,8.0,7.0,3.0,10.0,2.0,1.0,1.0,1,-1
124,795,6.0,12.0,8.0,7.0,3.0,10.0,2.0,1.0,1.0,1.0,10,-1
125,2108,12.0,8.0,7.0,3.0,10.0,2.0,1.0,1.0,1.0,10.0,8,-1
126,583,8.0,7.0,3.0,10.0,2.0,1.0,1.0,1.0,10.0,8.0,1,-1


In [13]:
"""
1. all_df2에는 csv별로 나눠진 뭉탱이 데이터프레임이 존재
2. all_df2리스트 안의 index 가 ??? 번째 csv라고 명칭

딕셔너리는
{
?? 번째 csv : 가장 자주나온 클래스, 
?? 번째 csv : 가장 자주나온 클래스,
?? 번째 csv : 가장 자주나온 클래스,
...
..
}
"""

dict_ = {}
for i in a:
    temp2 = all_df2[i]

    print('='*10)
    print(i)
    print('='*10)
    f = pd.merge(temp2, test[['id', 'target']], how='left', on='id')
    frequent = f['target'].value_counts()
    print(frequent)
    print('max:', f.max())
    dict_[i] = frequent.index[0]

0
3    84
0    32
2    12
Name: target, dtype: int64
max: id               3127.0
traffic(t-10)    1677.0
traffic(t-9)     1677.0
traffic(t-8)     1677.0
traffic(t-7)     1677.0
traffic(t-6)     1677.0
traffic(t-5)     1677.0
traffic(t-4)     1677.0
traffic(t-3)     1677.0
traffic(t-2)     1677.0
traffic(t-1)      982.0
traffic(t)        982.0
order              -1.0
target              3.0
dtype: float64
4
3    24
0    19
2     4
Name: target, dtype: int64
max: id               3102.0
traffic(t-10)    1019.0
traffic(t-9)     1019.0
traffic(t-8)     1019.0
traffic(t-7)     1019.0
traffic(t-6)     1019.0
traffic(t-5)     1019.0
traffic(t-4)     1019.0
traffic(t-3)     1019.0
traffic(t-2)     1019.0
traffic(t-1)     1019.0
traffic(t)       1019.0
order              -1.0
target              3.0
dtype: float64
10
2    59
1     2
3     1
Name: target, dtype: int64
max: id               3108.0
traffic(t-10)    6539.0
traffic(t-9)     6539.0
traffic(t-8)     6539.0
traffic(t-7)     6539.0
tra

In [14]:
temp2 = all_df2[8]
temp3 = pd.merge(temp2, test[['id', 'target']], how='left', on='id')
temp3.head(60)

,id,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),order,target
0,1968,11.0,3.0,8.0,18.0,10.0,14.0,21.0,1.0,4.0,6.0,1,-1,0
1,162,3.0,8.0,18.0,10.0,14.0,21.0,1.0,4.0,6.0,1.0,7,-1,0
2,12,8.0,18.0,10.0,14.0,21.0,1.0,4.0,6.0,1.0,7.0,13,-1,0
3,607,18.0,10.0,14.0,21.0,1.0,4.0,6.0,1.0,7.0,13.0,1,-1,0
4,1003,10.0,14.0,21.0,1.0,4.0,6.0,1.0,7.0,13.0,1.0,1,-1,0
5,1790,14.0,21.0,1.0,4.0,6.0,1.0,7.0,13.0,1.0,1.0,1,-1,0
6,2195,21.0,1.0,4.0,6.0,1.0,7.0,13.0,1.0,1.0,1.0,15,-1,0
7,2627,1.0,4.0,6.0,1.0,7.0,13.0,1.0,1.0,1.0,15.0,15,-1,0
8,984,4.0,6.0,1.0,7.0,13.0,1.0,1.0,1.0,15.0,15.0,20,-1,0
9,1119,6.0,1.0,7.0,13.0,1.0,1.0,1.0,15.0,15.0,20.0,14,-1,0


In [15]:
"""
딕셔너리 대로 가장 많은 횟수로 csv 클래스 바꾸기
"""
for d in dict_:
    temp2 = all_df2[d]
    temp3 = pd.merge(temp2, test[['id', 'target']], how='left', on='id')
    sub2.loc[temp3['id'], 'type'] = dict_[d]

In [16]:
sub2.to_csv('./result.csv', index=False) # 최종 submission 저장